# Deploy Model to online endpoint

- Models created with MLFlow do not require a scoring script nor an environment


In [54]:
import azure.ai.ml
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

print(azure.ai.ml._version.VERSION)

0.1.0b6


In [55]:
subscription_id = '5da07161-3770-4a4b-aa43-418cbbb627cf'
resource_group = 'aml-workspace-rg'
workspace = 'aml-workspace'

In [56]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [57]:
#connect to the workspace
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group,
        "workspace_name": workspace,
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/.azureml/config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f373dcb64f0>,
         subscription_id=5da07161-3770-4a4b-aa43-418cbbb627cf,
         resource_group_name=aml-workspace-rg,
         workspace_name=aml-workspace)


In [58]:
## Create Online Endpoint



In [59]:
### Configure endpoint

In [79]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="titanic online endpoint for mlflow model",
    auth_mode="key",
    tags={"oneline endpoint": "titanic"},
)

## Create endpoint

Using the MLClient created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.


In [80]:
ml_client.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-08272238724667.eastus.inference.ml.azure.com/score', 'swagger_uri': 'https://endpoint-08272238724667.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-08272238724667', 'description': 'titanic online endpoint for mlflow model', 'tags': {'oneline endpoint': 'titanic'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/aml-workspace-rg/providers/microsoft.machinelearningservices/workspaces/aml-workspace/onlineendpoints/endpoint-08272238724667', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:88449c92-2fa7-4734-8cc5-76e1776b55bb:753af4b4-759e-49c7-816e-572d8f5cb6e0?api-version=2022-02-01-preview'}, 'id': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb6

## Create deployment
A deployment is a set of resouces used for hosting the inferecing model using the *ManagedOnlineDeployment* class.  
Using the *ManagedOnlineDeployment* class, a developer can configure the following components

- name: name of the deployment
- endpoint_name: name of the endpoint to create the deployment under
- model: the model to use for the deployment
- instance_type: the VM side to use for deployment
- instance_count: the number of instances to use for the deployment
    

# Retrieve Model from registry

In [82]:
import mlflow
experiment = 'Chapter6'
current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

df = mlflow.search_runs([experiment_id])


run_id = df['run_id'].iloc[-1]
print(run_id)
print(type(run_id))

7685ed2a-881e-4e77-8c82-429623276f43
coral_gas_1hz6njncjl
<class 'str'>


In [84]:
import mlflow

print(run_id)
mlflow.set_experiment(experiment_name='Chapter6')
client = mlflow.tracking.MlflowClient()
client.list_artifacts(run_id=run_id)

coral_gas_1hz6njncjl


[<FileInfo: file_size=-1, is_dir=True, path='model'>,
 <FileInfo: file_size=-1, is_dir=True, path='system_logs'>,
 <FileInfo: file_size=-1, is_dir=False, path='test_confusion_matrix.png'>,
 <FileInfo: file_size=-1, is_dir=False, path='test_precision_recall_curve.png'>,
 <FileInfo: file_size=-1, is_dir=False, path='test_roc_curve.png'>,
 <FileInfo: file_size=-1, is_dir=False, path='training_confusion_matrix.png'>,
 <FileInfo: file_size=-1, is_dir=False, path='training_precision_recall_curve.png'>,
 <FileInfo: file_size=-1, is_dir=False, path='training_roc_curve.png'>,
 <FileInfo: file_size=-1, is_dir=True, path='user_logs'>]

In [86]:
import os
import shutil

file_path = client.download_artifacts(
    run_id, path="model"
)
shutil.copytree(file_path, './model', dirs_exist_ok=True)

'./model'

In [87]:
model = Model(path="./model/model.pkl")

## Get Environment

In [88]:
env = ml_client.environments.get(name = 'job_base_env', version = "1")

## Creating Scoring Script

score.py file

In [89]:
import os

# Create a folder for the experiment files
script_folder = 'ManagedOnlineEndpoint'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

ManagedOnlineEndpoint folder created


In [90]:
%%writefile $script_folder/score.py

import os 
import json
import joblib
from pandas import json_normalize
import pandas as pd

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    dict= json.loads(raw_data)
    df = json_normalize(dict['raw_data']) 
    y_pred = model.predict(df)
    print(type(y_pred))
    
    result = {"result": y_pred.tolist()}
    return result

Overwriting ManagedOnlineEndpoint/score.py


## Configure the deployment

- retrieve the experiment id for this run, and the run id to retrieve the model from the registered model list

In [95]:
import datetime


blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment="azureml:job_base_env:1",
    code_configuration=CodeConfiguration(
        code="./ManagedOnlineEndpoint", scoring_script="score.py"
    ),
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

## Create deployment

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint endpoint-08272238724667 exists
Uploading model.pkl (< 1 MB): 100%|██████████| 2.61k/2.61k [00:00<00:00, 140kB/s]


Creating/updating online deployment blue 

.............

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint)

In [ ]:
## Get Endpoint details

# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(endpoint)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)


In [ ]:
import json
url = endpoint.scoring_uri
api_key = endpoint.key  # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
import requests

def MakePrediction(df):
    endpoint_url = url
    body = df.to_json(orient='records') 
    body = '{"raw_data": ' + body + '}'
    print(body)
    r = requests.post(endpoint_url, headers=headers, data=body)
    return (r.json())


dataset = Dataset.get_by_name(ws, name='Titanic-tabular-dataset')
df = dataset.to_pandas_dataframe().head(5)
dftest = df.drop(['Survived'], axis=1)

results = MakePrediction(dftest)

val = results['result']
print('')
print('predictions')
print(val)